In [1]:
from ctypes import *
import time
from sys import path
import sys
from os import sep
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

if sys.platform.startswith("win"):
    dwf = cdll.dwf
    constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
    constants_path = "/Applications/WaveForms.app/Contents/Resources/SDK/samples/py"
else:
    dwf = cdll.LoadLibrary("libdwf.so")
    constants_path = "/usr/share/digilent/waveforms/samples/py"

path.append(constants_path)
import dwfconstants as constants

In [2]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0))
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.00V


In [3]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
ID_REG = 0x92
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [4]:
write_register(ENABLE_REG, 0x01)
time.sleep(0.01)
write_register(ENABLE_REG, 0x03)
write_register(ATIME_REG, 0xFF)
write_register(CONTROL_REG, 0x02)

def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [ ]:
data1={'int_time': [], 'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(ATIME_REG, 0xFF) # Integration Time value FF
write_register(CONTROL_REG, 0x02) # Gain 4x
freq_arr = [1, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000, 400000, 700000, 1000000]

for freq_values in freq_arr:
    dwf.FDwfDigitalI2cRateSet(hdwf, c_double(freq_values))
    id = read_register(ID_REG)
    if id == 0x44 or id == 0x4D:
        print(f"Frequency: {freq_values} Hz, ID: {id:#04x} - Device Found")
    else:
        print(f"Frequency: {freq_values} Hz - Device Not Found")

Frequency: 1 Hz - Device Not Found
Frequency: 10 Hz - Device Not Found
Frequency: 50 Hz - Device Not Found
Frequency: 100 Hz - Device Not Found
Frequency: 500 Hz - Device Not Found
Frequency: 1000 Hz, ID: 0x4d - Device Found
Frequency: 5000 Hz, ID: 0x4d - Device Found
Frequency: 10000 Hz, ID: 0x4d - Device Found
Frequency: 50000 Hz, ID: 0x4d - Device Found
Frequency: 100000 Hz, ID: 0x4d - Device Found
Frequency: 400000 Hz, ID: 0x4d - Device Found
Frequency: 700000 Hz, ID: 0x4d - Device Found
Frequency: 1000000 Hz, ID: 0x4d - Device Found


In [9]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)
cleanup()